### The Lifecycle Configuration Script downloads the jar file to the jdbc directory under home directory (~)

In [ ]:
jar_dir <- "~/SageMaker/jdbc"
jar_file <- list.files(jar_dir, pattern = "Athena")
jar_dir_file <- file.path(jar_dir,jar_file)
libjvm_path <- system("find /usr/lib/jvm/ -name libjvm.so", intern=TRUE)
libjvm_path
dyn.load(libjvm_path)

options( java.parameters = "-Xmx8g" )

### Replace the s3_bucket with the value of *SageMakerRS3BucketName* in the CloudFormation Outputs

* For example:-
 * *s3_bucket <- "blog-sagemaker-r-sagemakerrdatabucket-xxxxxxxxxxxxx"*

In [ ]:
s3_bucket <- "<value of SageMakerRS3BucketName in the CloudFormation Outputs>"

In [ ]:
session_token <- system("curl -X PUT 'http://169.254.169.254/latest/api/token' -H 'X-aws-ec2-metadata-token-ttl-seconds: 21600'",intern=TRUE)
session_token
region_lookup_cmd <- paste( "curl -H \"X-aws-ec2-metadata-token: ", session_token, "\"  -s http://169.254.169.254/latest/dynamic/instance-identity/document | jq .region -r")
region <-  system(region_lookup_cmd,intern=TRUE)
jdbc_string <- paste("jdbc:awsathena://AwsRegion=",region,";S3OutputLocation=s3://",s3_bucket,"/;AwsCredentialsProviderClass=com.simba.athena.amazonaws.auth.DefaultAWSCredentialsProviderChain",sep="")
jdbc_string


In [ ]:
library(DBI)
library(rJava)
library(RJDBC)
options( java.parameters = "-Xmx8g" )
drv <- JDBC("com.simba.athena.jdbc.Driver",jar_dir_file,identifier.quote="`")
conn <- dbConnect(drv, jdbc_string)
conn

In [ ]:
show_databases <- dbGetQuery(conn, "show databases")
show_databases

In [ ]:
dbListObjects(conn)

### Modify or replace with valid date ranges in the SELECT query

In [ ]:
fetch <- dbGetQuery(conn,"SELECT os, COUNT(*) count FROM mydatabase.cloudfront_logs WHERE date BETWEEN date '2014-07-05' AND date '2014-08-05' GROUP BY os;")

In [ ]:
fetch